<a href="https://colab.research.google.com/github/YantCaccia/Tirocinio/blob/main/GRID_VSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Unzip dataset from Drive

In [ ]:
!rm -rf "/content/sample_data"
!rm -rf "/content/myFinalDataset"
!rm -rd "/content/myDataset"
!unzip -qq "/content/drive/MyDrive/tirocinioWorkingDirectory/datasets/grid/finalDataset/myFinalDatasetCroppedNewEdition.zip" -d "/content/"

I'm stupid so let's move the files in the correct directory

In [ ]:
!mv "/content/content/myDataset/" "/content/myDataset"
!rm -rf "/content/content"

In [ ]:
#tbd
#test if file removing worked

!find "/content/myDataset/again" -name "WORDagainSP8SENbgag6a.mpg"

#Video generator

In [ ]:
!pip install keras-video-generators
#the following line is mandatory to solve a bug in the current release of keras-video-generators library
!sed -i '18s/.*/from tensorflow.keras.utils import Sequence/' "/usr/local/lib/python3.7/dist-packages/keras_video/generator.py"

In [ ]:
import os
import gc
import glob
from keras_video import VideoFrameGenerator
import keras_video.utils
import tensorflow
from tensorflow import keras

# use sub directories names as classes
classes = [i.split(os.path.sep)[3] for i in glob.glob('/content/myDataset/*')]
classes.sort()

# some global params
SIZE = (80, 40) #(360, 288)
CHANNELS = 1
NBFRAME = 3
BS =  64

# pattern to get videos and classes
glob_pattern='/content/myDataset/{classname}/*'

# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.15, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)


valid = train.get_validation_generator()

# Model creation and training

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D, Flatten, GRU, Dropout, TimeDistributed, Dense

def build_convnet(shape=(112, 112, 3)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
model = action_model(INSHAPE, len(classes))
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)

EPOCHS=10

# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
path = "/content/chkp"
if not os.path.exists(path):
  os.mkdir(path)

class ClearMemory(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        keras.backend.clear_session()

callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
    keras.callbacks.EarlyStopping(patience=7),
    ClearMemory()
]

model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
!cp -r "/content/chkp" "/content/drive/MyDrive/tirocinioWorkingDirectory/datasets/grid/weights/1509"

# Model creation and training (TBR, just to quickly test the library)

In [ ]:
import gc
import tensorflow as tf
from tensorflow.keras.layers import ConvLSTM2D, Dropout, Flatten, Dense, SimpleRNN

class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()

model = tf.keras.Sequential()
#model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (NBFRAME, SIZE[1], SIZE[0], CHANNELS)))
model.add(SimpleRNN(units = 64, return_sequences = False, input_shape = (NBFRAME, SIZE[1], SIZE[0], CHANNELS)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
#model.add(Dropout(0.3))
model.add(Dense(128, activation = "relu"))
#model.add(Dropout(0.3))
model.add(Dense(51, activation = "relu"))

opt = keras.optimizers.Adam(0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"], run_eagerly=True)

earlystop = tf.keras.callbacks.EarlyStopping(patience=7)
callbacks = [earlystop, ClearMemory()]

history = model.fit(x = train, epochs=10, batch_size = BS, callbacks=callbacks)
